In [80]:
import os
import random

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms
from torch.utils.data import DataLoader,TensorDataset
from telescope_torch import *
import time
import copy
import numpy as np

In [15]:
arrange =  np.array([28,29,30,31,0,4,8,12,
                     24,25,26,27,1,5,9,13,
                     20,21,22,23,2,6,10,14,
                     16,17,18,19,3,7,11,15,
                     47,43,39,35,35,34,33,32,
                     46,42,38,34,39,38,37,36,
                     45,41,37,33,43,42,41,40,
                     44,40,36,32,47,46,45,44])

arrMask= np.array([1,1,1,1,1,1,1,1,
                   1,1,1,1,1,1,1,1,
                   1,1,1,1,1,1,1,1,
                   1,1,1,1,1,1,1,1,
                   1,1,1,1,0,0,0,0,
                   1,1,1,1,0,0,0,0,
                   1,1,1,1,0,0,0,0,
                   1,1,1,1,0,0,0,0,])
    
calQMask=np.array([1,1,1,1,1,1,1,1,
                    1,1,1,1,1,1,1,1,
                    1,1,1,1,1,1,1,1,
                    1,1,1,1,1,1,1,1,
                    1,1,1,1,0,0,0,0,
                    1,1,1,1,0,0,0,0,
                    1,1,1,1,0,0,0,0,
                    1,1,1,1,0,0,0,0,])
undo = [4,12,20,28,
 5,13,21,29,6,14,
 22,30,7,15,23,31,
 24,25,26,27,16,17,
 18,19,8,9,10,11,0,
 1,2,3,59,51,43,35,
 58,50,42,34,57,49,
 41,33,56,48,40,32]


def prepInput(normData):
    
    shape = (1,8,8)
    inputdata = normData[:,arrange]

    inputdata[:,arrMask==0]=0  #zeros out repeated entries

    shaped_data = inputdata.reshape(len(inputdata),shape[0],shape[1],shape[2])
    
    return shaped_data


In [3]:
# types = [1,
#          2,
#         3]
# max_size = 75000
# for ty in types:
    

In [97]:
def get_data(data,cur_model):
        cur_data = []
        for d in data:
            if d[1] == cur_model:
                cur_data = d[0]
                break
        return cur_data
def prepare_dataloader(ty, 
                       train_batch_size=128,
                       eval_batch_size=256,
                       max_size = 1000,
                      threshold = 0,device = 'cuda'):
    
    print(ty)
    

    path = 'MIT_TTbar'
    prefixed = [filename for filename in os.listdir(path) if filename.startswith("dt_norm")]

    data = []
    for p in prefixed:
        data.append([torch.load(f'{path}/{p}'),p])
    path = '/uscms/home/nswood/nobackup/Notebooks/AE_Dev/models/batched_models/wafer_layer_split_mip_std_1_mean_nonzero'

    models = os.listdir(path)[9:]

    #all past 29
    cur_train = []
    
    for m in models:
        if int(m[8]) == ty:
            cur_train.append(m)
    print(cur_train)
    
    #Calculating MSE of trained models
    path = '/uscms/home/nswood/nobackup/Notebooks/AE_Dev/models/batched_models/wafer_layer_split_mip_std_1_mean_nonzero'

    cur_model = models[1]

    

    test_set = []
    train_set = []
    pred_set = []
    
    for m in cur_train:
        cur_model = torch.load(os.path.join(path,m)).to('cuda')
        cur_data = get_data(data,m).to('cuda')
        if len(cur_data)< max_size:
            train_set.append(cur_data[0:int(0.8*len(cur_data))])
            test_data = cur_data[-int(0.19*len(cur_data)):]
            test_set.append(test_data)
        else:
            train_set.append(cur_data[0:int(0.8*max_size)])

            test_data = cur_data[-int(0.19*max_size):]
            test_set.append(test_data)

    
    device = 'cuda'
    
    train_dt = torch.vstack(train_set).to(device)
    test_dt = torch.vstack(test_set).to(device)    
    mean = 0
    std = 1
    size_1_train =len(train_dt)
    size_1_test = len(test_dt)
    
    
    
    train_loc = train_dt
    test_loc = test_dt
    # train = torch.vstack([train,largest[0:1000]] )
    train_loc=train_loc[torch.randperm(train_loc.size()[0])]
    test_loc=test_loc[torch.randperm(test_loc.size()[0])]
    train_loc_d1_flat = DataLoader(
        TensorDataset(torch.Tensor(train_loc)),
        batch_size=train_batch_size,
        shuffle=False
    )

    test_loc_d1_flat = DataLoader(
        TensorDataset(torch.Tensor(test_loc)),
        batch_size=100,
        shuffle=False
    )

    train_1 = train_dt[:,0:48]
    test_1 = test_dt[:,0:48]
    train_1=train_1[torch.randperm(train_1.size()[0])]
    test_1=test_1[torch.randperm(test_1.size()[0])]
    print(len(train_1))

    train_tc_sum = torch.unsqueeze(torch.sum(train_1[:,0:48],dim = 1),dim=1)
    test_tc_sum = torch.unsqueeze(torch.sum(test_1[:,0:48],dim = 1),dim=1)


    train_1 = train_1[torch.squeeze(train_tc_sum >threshold,dim=1)]
    test_1 = test_1[torch.squeeze(test_tc_sum >threshold,dim=1)]
    
    sum_TC_test = torch.unsqueeze(torch.sum(test_1[:,0:48],dim=1),dim=1)

    train_tc_sum = torch.unsqueeze(train_tc_sum[train_tc_sum >threshold],dim=1)
    test_tc_sum = torch.unsqueeze(test_tc_sum[test_tc_sum >threshold],dim=1)

    train_1_d1_flat = DataLoader(
        TensorDataset(torch.Tensor(train_1),train_tc_sum),
        batch_size=train_batch_size,
        shuffle=False
    )

    test_1_d1_flat = DataLoader(
        TensorDataset(torch.Tensor(test_1),test_tc_sum),
        batch_size=eval_batch_size,
        shuffle=False
    )
    
    

    return train_1_d1_flat, test_1_d1_flat

In [98]:
print(ty)
num_workers = 1 
train_batch_size=128
eval_batch_size=256
max_size = 1000
threshold = 0
device = 'cuda'
path = 'MIT_TTbar'
prefixed = [filename for filename in os.listdir(path) if filename.startswith("dt_norm")]

data = []
for p in prefixed:
    data.append([torch.load(f'{path}/{p}'),p])
path = '/uscms/home/nswood/nobackup/Notebooks/AE_Dev/models/batched_models/wafer_layer_split_mip_std_1_mean_nonzero'

models = os.listdir(path)[9:]

#all past 29
cur_train = []

for m in models:
    if int(m[8]) == ty:
        cur_train.append(m)
print(cur_train)

#Calculating MSE of trained models
path = '/uscms/home/nswood/nobackup/Notebooks/AE_Dev/models/batched_models/wafer_layer_split_mip_std_1_mean_nonzero'

cur_model = models[1]



test_set = []
train_set = []
pred_set = []

for m in cur_train:
    cur_model = torch.load(os.path.join(path,m)).to('cuda')
    cur_data = get_data(data,m).to('cuda')
    if len(cur_data)< max_size:
        train_set.append(cur_data[0:int(0.8*len(cur_data))])
        test_data = cur_data[-int(0.19*len(cur_data)):]
        test_set.append(test_data)
    else:
        train_set.append(cur_data[0:int(0.8*max_size)])

        test_data = cur_data[-int(0.19*max_size):]
        test_set.append(test_data)


device = 'cuda'

train_dt = torch.vstack(train_set).to(device)
test_dt = torch.vstack(test_set).to(device)    
mean = 0
std = 1
size_1_train =len(train_dt)
size_1_test = len(test_dt)



train_loc = train_dt
test_loc = test_dt
# train = torch.vstack([train,largest[0:1000]] )
train_loc=train_loc[torch.randperm(train_loc.size()[0])]
test_loc=test_loc[torch.randperm(test_loc.size()[0])]
train_loc_d1_flat = DataLoader(
    TensorDataset(torch.Tensor(train_loc)),
    batch_size=train_batch_size,
    shuffle=False
)

test_loc_d1_flat = DataLoader(
    TensorDataset(torch.Tensor(test_loc)),
    batch_size=100,
    shuffle=False
)

train_1 = train_dt[:,0:48]
test_1 = test_dt[:,0:48]
train_1=train_1[torch.randperm(train_1.size()[0])]
test_1=test_1[torch.randperm(test_1.size()[0])]
print(len(train_1))

train_tc_sum = torch.unsqueeze(torch.sum(train_1[:,0:48],dim = 1),dim=1)
test_tc_sum = torch.unsqueeze(torch.sum(test_1[:,0:48],dim = 1),dim=1)


train_1 = train_1[torch.squeeze(train_tc_sum >threshold,dim=1)]
test_1 = test_1[torch.squeeze(test_tc_sum >threshold,dim=1)]

sum_TC_test = torch.unsqueeze(torch.sum(test_1[:,0:48],dim=1),dim=1)

train_tc_sum = torch.unsqueeze(train_tc_sum[train_tc_sum >threshold],dim=1)
test_tc_sum = torch.unsqueeze(test_tc_sum[test_tc_sum >threshold],dim=1)

train_1_d1_flat = DataLoader(
    TensorDataset(torch.Tensor(train_1),train_tc_sum),
    batch_size=train_batch_size,
    shuffle=False
)

test_1_d1_flat = DataLoader(
    TensorDataset(torch.Tensor(test_1),test_tc_sum),
    batch_size=eval_batch_size,
    shuffle=False
)

1
['dt_norm_1_1', 'dt_norm_1_3', 'dt_norm_1_5', 'dt_norm_1_7', 'dt_norm_1_9', 'dt_norm_1_11', 'dt_norm_1_13', 'dt_norm_1_15', 'dt_norm_1_17', 'dt_norm_1_19', 'dt_norm_1_21', 'dt_norm_1_23', 'dt_norm_1_25', 'dt_norm_1_27', 'dt_norm_1_29', 'dt_norm_1_30', 'dt_norm_1_31', 'dt_norm_1_32', 'dt_norm_1_33']
15200


In [106]:
def evaluate_model(model, test_loader, device, criterion=None):

    model.eval()
    model.to(device)
    model.encoder.eval()
    model.encoder.to(device)
    model.encoder.device = device
    model.decoder.eval()
    model.decoder.device = device
    model.decoder.to(device)

    running_loss = 0
    

    for i, (data,sum_TC) in enumerate(test_loader):

        outputs = model(prepInput(data[:,0:48]),sum_TC)
        _, preds = torch.max(outputs, 1)

        
        loss = criterion(outputs, prepInput(data)).item()
        

        # statistics
        running_loss += loss

    eval_loss = running_loss / len(test_loader.dataset)

    return eval_loss

In [121]:
def train_model(model, train_loader, test_loader, device, learning_rate=1e-1, num_epochs=200):

    # The training configurations were not carefully selected.

    criterion = telescopeMSE8x8

    model.to(device)
    model.to(device)
    model.encoder.eval()
    model.encoder.to(device)
    model.encoder.device = device
    model.decoder.eval()
    model.decoder.device = device
    model.decoder.to(device)
    
    optimizer = optim.Adam(model.parameters(), learning_rate,weight_decay=0)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min',factor = 0.5)

    # Evaluation
    model.eval()
    eval_loss = evaluate_model(model=model, test_loader=test_loader, device=device, criterion=criterion)
    print("Epoch: {:02d} Eval Loss: {:.3f}".format(-1, eval_loss))

    for epoch in range(num_epochs):

        # Training
        model.train()

        running_loss = 0

        for i, (data,sum_TC) in enumerate(train_loader):

            data = data.to(device)
            sum_TC = sum_TC.to(device)
            
            # zero the parameter gradients
            optimizer.zero_grad()
            
            masked_dt = prepInput(data[:,0:48])
            # forward + backward + optimize
            outputs = model(masked_dt,sum_TC)
            loss = criterion(outputs, masked_dt)
            loss.backward()
            optimizer.step()

            # statistics
            running_loss += loss.item() 

        train_loss = running_loss / len(train_loader.dataset)

        # Evaluation
        model.eval()
        eval_loss = evaluate_model(model=model, test_loader=test_loader, device=device, criterion=criterion)

        # Set learning rate scheduler
        scheduler.step(loss.item())

        print("Epoch: {:03d} Train Loss: {:.3f} Eval Loss: {:.3f} ".format(epoch, train_loss, eval_loss))

    return model

In [7]:
def calibrate_model(model, loader, device=torch.device("cpu:0")):

    model.to(device)
    model.eval()

    for data,sum_TC in loader:
        data = data.to(device)
        sum_TC = sum_TC.to(device)
        masked_dt = prepInput(data[:,0:48])
        _ = model(masked_dt,sum_TC)


In [8]:
def measure_inference_latency(model,
                              device,
                              input_size=(1,8,8),
                              num_samples=100,
                              num_warmups=10):

    model.to(device)
    model.eval()

    x = torch.rand(size=input_size).to(device)
    sum_x = t
    with torch.no_grad():
        for _ in range(num_warmups):
            _ = model(x)
    torch.cuda.synchronize()

    with torch.no_grad():
        start_time = time.time()
        for _ in range(num_samples):
            _ = model(x)
            torch.cuda.synchronize()
        end_time = time.time()
    elapsed_time = end_time - start_time
    elapsed_time_ave = elapsed_time / num_samples

    return elapsed_time_ave

def save_model(model, model_dir, model_filename):

    if not os.path.exists(model_dir):
        os.makedirs(model_dir)
    model_filepath = os.path.join(model_dir, model_filename)
    torch.save(model.state_dict(), model_filepath)

def load_model(model, model_filepath, device):

    model.load_state_dict(torch.load(model_filepath, map_location=device))

    return model

def save_torchscript_model(model, model_dir, model_filename):

    if not os.path.exists(model_dir):
        os.makedirs(model_dir)
    model_filepath = os.path.join(model_dir, model_filename)
    torch.jit.save(torch.jit.script(model), model_filepath)

def load_torchscript_model(model_filepath, device):

    model = torch.jit.load(model_filepath, map_location=device)

    return model

In [9]:
def create_model(num_classes=10):

    # The number of channels in ResNet18 is divisible by 8.
    # This is required for fast GEMM integer matrix multiplication.
    # model = torchvision.models.resnet18(pretrained=False)
    model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', pretrained=True)

    # We would use the pretrained ResNet18 as a feature extractor.
    # for param in model.parameters():
    #     param.requires_grad = False
    
    # Modify the last FC layer
    # num_features = model.fc.in_features
    # model.fc = nn.Linear(num_features, 10)

    return model

class QuantizedResNet18(nn.Module):
    def __init__(self, model_fp32):
        super(QuantizedResNet18, self).__init__()
        # QuantStub converts tensors from floating point to quantized.
        # This will only be used for inputs.
        self.quant = torch.quantization.QuantStub()
        # DeQuantStub converts tensors from quantized to floating point.
        # This will only be used for outputs.
        self.dequant = torch.quantization.DeQuantStub()
        # FP32 model
        self.model_fp32 = model_fp32

    def forward(self, x):
        # manually specify where tensors will be converted from floating
        # point to quantized in the quantized model
        x = self.quant(x)
        x = self.model_fp32(x)
        # manually specify where tensors will be converted from quantized
        # to floating point in the quantized model
        x = self.dequant(x)
        return x

def model_equivalence(model_1, model_2, device, rtol=1e-05, atol=1e-08, num_tests=100, input_size=(1,3,32,32)):

    model_1.to(device)
    model_2.to(device)

    for _ in range(num_tests):
        x = torch.rand(size=input_size).to(device)
        y1 = model_1(x).detach().cpu().numpy()
        y2 = model_2(x).detach().cpu().numpy()
        if np.allclose(a=y1, b=y2, rtol=rtol, atol=atol, equal_nan=False) == False:
            print("Model equivalence test sample failed: ")
            print(y1)
            print(y2)
            return False

    return True

In [137]:
ty = 1
out_dir = 'models/CAE_Quant'
model_name = f'CAE_Quant_large_{ty}'

In [139]:
torch.save(quantized_model.state_dict(), os.path.join(out_dir, model_name))

In [143]:
import importlib
import QuantizedAE
import CAE_encoders
import CAE_decoders
importlib.reload(QuantizedAE)
lr = 5e-5

num_epochs = 200

max_size = 70000

for ty in [1,2,3]:
    
    out_dir = 'models/CAE_Quant'
    model_name = f'CAE_Quant_large_{ty}'
    
    cuda_device = torch.device("cuda:0")
    cpu_device = torch.device("cpu:0")

    model_dir = "models/CAE_Quant"
    model_filename = "resnet18_cifar10.pt"
    quantized_model_filename = "resnet18_quantized_cifar10.pt"
    model_filepath = os.path.join(model_dir, model_filename)
    quantized_model_filepath = os.path.join(model_dir, quantized_model_filename)

    train_loader, test_loader = prepare_dataloader(ty,max_size = max_size,  train_batch_size=500, eval_batch_size=500)

    enc = CAE_encoders.CAE_conv_encoder_arch(16)

    quantization_config = torch.quantization.get_default_qconfig("fbgemm")
    enc.qconfig = quantization_config
    print(enc.qconfig)
    torch.quantization.prepare_qat(enc, inplace=True)
    quantized_model = QuantizedAE.QuantizedAE(enc, CAE_decoders.CAE_conv_large_decoder_arch(16))
    # # Use training data for calibration.
    print("Training QAT Model...")
    quantized_model.train()
    train_model(model=quantized_model, train_loader=train_loader, test_loader=test_loader, device=cuda_device, learning_rate=lr, num_epochs=num_epochs)
    quantized_model.to(cpu_device)
    torch.save(quantized_model.state_dict(), os.path.join(out_dir, model_name))

1
['dt_norm_1_1', 'dt_norm_1_3', 'dt_norm_1_5', 'dt_norm_1_7', 'dt_norm_1_9', 'dt_norm_1_11', 'dt_norm_1_13', 'dt_norm_1_15', 'dt_norm_1_17', 'dt_norm_1_19', 'dt_norm_1_21', 'dt_norm_1_23', 'dt_norm_1_25', 'dt_norm_1_27', 'dt_norm_1_29', 'dt_norm_1_30', 'dt_norm_1_31', 'dt_norm_1_32', 'dt_norm_1_33']
1057701
QConfig(activation=functools.partial(<class 'torch.ao.quantization.observer.HistogramObserver'>, reduce_range=True){}, weight=functools.partial(<class 'torch.ao.quantization.observer.PerChannelMinMaxObserver'>, dtype=torch.qint8, qscheme=torch.per_channel_symmetric){})
Training QAT Model...
Epoch: -1 Eval Loss: 3.075
Epoch: 000 Train Loss: 0.814 Eval Loss: 0.301 
Epoch: 001 Train Loss: 0.249 Eval Loss: 0.218 
Epoch: 002 Train Loss: 0.208 Eval Loss: 0.198 
Epoch: 003 Train Loss: 0.191 Eval Loss: 0.184 
Epoch: 004 Train Loss: 0.180 Eval Loss: 0.174 
Epoch: 005 Train Loss: 0.172 Eval Loss: 0.170 
Epoch: 006 Train Loss: 0.167 Eval Loss: 0.165 
Epoch: 007 Train Loss: 0.164 Eval Loss: 0.

Epoch: 161 Train Loss: 0.091 Eval Loss: 0.090 
Epoch: 162 Train Loss: 0.091 Eval Loss: 0.090 
Epoch: 163 Train Loss: 0.091 Eval Loss: 0.090 
Epoch: 164 Train Loss: 0.091 Eval Loss: 0.090 
Epoch: 165 Train Loss: 0.091 Eval Loss: 0.090 
Epoch: 166 Train Loss: 0.091 Eval Loss: 0.090 
Epoch: 167 Train Loss: 0.091 Eval Loss: 0.090 
Epoch: 168 Train Loss: 0.091 Eval Loss: 0.090 
Epoch: 169 Train Loss: 0.091 Eval Loss: 0.090 
Epoch: 170 Train Loss: 0.091 Eval Loss: 0.090 
Epoch: 171 Train Loss: 0.091 Eval Loss: 0.090 
Epoch: 172 Train Loss: 0.091 Eval Loss: 0.090 
Epoch: 173 Train Loss: 0.091 Eval Loss: 0.090 
Epoch: 174 Train Loss: 0.091 Eval Loss: 0.090 
Epoch: 175 Train Loss: 0.090 Eval Loss: 0.090 
Epoch: 176 Train Loss: 0.090 Eval Loss: 0.090 
Epoch: 177 Train Loss: 0.090 Eval Loss: 0.090 
Epoch: 178 Train Loss: 0.090 Eval Loss: 0.090 
Epoch: 179 Train Loss: 0.090 Eval Loss: 0.090 
Epoch: 180 Train Loss: 0.090 Eval Loss: 0.090 
Epoch: 181 Train Loss: 0.090 Eval Loss: 0.090 
Epoch: 182 Tr

Epoch: 119 Train Loss: 0.053 Eval Loss: 0.050 
Epoch: 120 Train Loss: 0.053 Eval Loss: 0.050 
Epoch: 121 Train Loss: 0.053 Eval Loss: 0.050 
Epoch: 122 Train Loss: 0.053 Eval Loss: 0.050 
Epoch: 123 Train Loss: 0.052 Eval Loss: 0.050 
Epoch: 124 Train Loss: 0.053 Eval Loss: 0.050 
Epoch: 125 Train Loss: 0.053 Eval Loss: 0.050 
Epoch: 126 Train Loss: 0.053 Eval Loss: 0.050 
Epoch: 127 Train Loss: 0.052 Eval Loss: 0.050 
Epoch: 128 Train Loss: 0.052 Eval Loss: 0.050 
Epoch: 129 Train Loss: 0.052 Eval Loss: 0.050 
Epoch: 130 Train Loss: 0.053 Eval Loss: 0.049 
Epoch: 131 Train Loss: 0.052 Eval Loss: 0.049 
Epoch: 132 Train Loss: 0.052 Eval Loss: 0.049 
Epoch: 133 Train Loss: 0.052 Eval Loss: 0.049 
Epoch: 134 Train Loss: 0.052 Eval Loss: 0.049 
Epoch: 135 Train Loss: 0.052 Eval Loss: 0.049 
Epoch: 136 Train Loss: 0.052 Eval Loss: 0.049 
Epoch: 137 Train Loss: 0.052 Eval Loss: 0.049 
Epoch: 138 Train Loss: 0.052 Eval Loss: 0.049 
Epoch: 139 Train Loss: 0.052 Eval Loss: 0.049 
Epoch: 140 Tr

Epoch: 076 Train Loss: 0.399 Eval Loss: 0.397 
Epoch: 077 Train Loss: 0.399 Eval Loss: 0.396 
Epoch: 078 Train Loss: 0.399 Eval Loss: 0.396 
Epoch: 079 Train Loss: 0.399 Eval Loss: 0.405 
Epoch: 080 Train Loss: 0.399 Eval Loss: 0.399 
Epoch: 081 Train Loss: 0.399 Eval Loss: 0.397 
Epoch: 082 Train Loss: 0.399 Eval Loss: 0.400 
Epoch: 083 Train Loss: 0.398 Eval Loss: 0.395 
Epoch: 084 Train Loss: 0.399 Eval Loss: 0.404 
Epoch: 085 Train Loss: 0.398 Eval Loss: 0.397 
Epoch: 086 Train Loss: 0.398 Eval Loss: 0.396 
Epoch: 087 Train Loss: 0.398 Eval Loss: 0.396 
Epoch: 088 Train Loss: 0.398 Eval Loss: 0.396 
Epoch: 089 Train Loss: 0.398 Eval Loss: 0.395 
Epoch: 090 Train Loss: 0.398 Eval Loss: 0.397 
Epoch: 091 Train Loss: 0.398 Eval Loss: 0.396 
Epoch: 092 Train Loss: 0.398 Eval Loss: 0.396 
Epoch: 093 Train Loss: 0.398 Eval Loss: 0.395 
Epoch: 094 Train Loss: 0.397 Eval Loss: 0.402 
Epoch: 095 Train Loss: 0.397 Eval Loss: 0.395 
Epoch: 096 Train Loss: 0.397 Eval Loss: 0.396 
Epoch: 097 Tr

In [144]:
!ls models/CAE_Quant

CAE_Quant_large_1  CAE_Quant_large_2  CAE_Quant_large_3


In [104]:


# Using high-level static quantization wrapper
# The above steps, including torch.quantization.prepare, calibrate_model, and torch.quantization.convert, are also equivalent to
# quantized_model = torch.quantization.quantize_qat(model=quantized_model, run_fn=train_model, run_args=[train_loader, test_loader, cuda_device], mapping=None, inplace=False)

quantized_model = torch.quantization.convert(quantized_model, inplace=True)

quantized_model.eval()

# Print quantized model.
print(quantized_model)

# Save quantized model.
save_torchscript_model(model=quantized_model, model_dir=model_dir, model_filename=quantized_model_filename)

# Load quantized model.
quantized_jit_model = load_torchscript_model(model_filepath=quantized_model_filepath, device=cpu_device)

_, int8_eval_accuracy = evaluate_model(model=quantized_jit_model, test_loader=test_loader, device=cpu_device, criterion=None)

print("INT8 evaluation accuracy: {:.3f}".format(int8_eval_accuracy))

int8_cpu_inference_latency = measure_inference_latency(model=quantized_model, device=cpu_device, input_size=(1,3,32,32), num_samples=100)
int8_jit_cpu_inference_latency = measure_inference_latency(model=quantized_jit_model, device=cpu_device, input_size=(1,3,32,32), num_samples=100)

print("FP32 CPU Inference Latency: {:.2f} ms / sample".format(fp32_cpu_inference_latency * 1000))
print("FP32 CUDA Inference Latency: {:.2f} ms / sample".format(fp32_gpu_inference_latency * 1000))
print("INT8 CPU Inference Latency: {:.2f} ms / sample".format(int8_cpu_inference_latency * 1000))
print("INT8 JIT CPU Inference Latency: {:.2f} ms / sample".format(int8_jit_cpu_inference_latency * 1000))

/uscms_data/d3/nswood/mambaforge/envs/myenv/lib/python3.10/site-packages/torch/ao/quantization/observer.py:1204: UserWarning: must run observer before calling calculate_qparams.                                    Returning default scale and zero point 
  warnings.warn(


RuntimeError: Unsupported qscheme: per_channel_affine

In [29]:
def main():
    ty = 1
    random_seed = 0
    num_classes = 10
    cuda_device = torch.device("cuda:0")
    cpu_device = torch.device("cpu:0")

    model_dir = "models/CAE_Quant"
    model_filename = "resnet18_cifar10.pt"
    quantized_model_filename = "resnet18_quantized_cifar10.pt"
    model_filepath = os.path.join(model_dir, model_filename)
    quantized_model_filepath = os.path.join(model_dir, quantized_model_filename)

#     set_random_seeds(random_seed=random_seed)

    # Create an untrained model.
    #model = create_model(num_classes=num_classes)

    train_loader, test_loader = prepare_dataloader(ty, num_workers=8, train_batch_size=128, eval_batch_size=256)
    if False:
        # Train model.
        print("Training Model...")
        model = train_model(model=model, train_loader=train_loader, test_loader=test_loader, device=cuda_device, learning_rate=1e-1, num_epochs=200)
        # Save model.
        save_model(model=model, model_dir=model_dir, model_filename=model_filename)
        # Load a pretrained model.
        model = load_model(model=model, model_filepath=model_filepath, device=cuda_device)
        # Move the model to CPU since static quantization does not support CUDA currently.
        model.to(cpu_device)
        # Make a copy of the model for layer fusion
        fused_model = copy.deepcopy(model)

        model.train()
        # The model has to be switched to training mode before any layer fusion.
        # Otherwise the quantization aware training will not work correctly.
        fused_model.train()

        # Fuse the model in place rather manually.
        fused_model = torch.quantization.fuse_modules(fused_model, [["conv1", "bn1", "relu"]], inplace=True)
        for module_name, module in fused_model.named_children():
            if "layer" in module_name:
                for basic_block_name, basic_block in module.named_children():
                    torch.quantization.fuse_modules(basic_block, [["conv1", "bn1", "relu1"], ["conv2", "bn2"]], inplace=True)
                    for sub_block_name, sub_block in basic_block.named_children():
                        if sub_block_name == "downsample":
                            torch.quantization.fuse_modules(sub_block, [["0", "1"]], inplace=True)

        # Print FP32 model.
        print(model)
        # Print fused model.
        print(fused_model)

        # Model and fused model should be equivalent.
        model.eval()
        fused_model.eval()
        assert model_equivalence(model_1=model, model_2=fused_model, device=cpu_device, rtol=1e-03, atol=1e-06, num_tests=100, input_size=(1,3,32,32)), "Fused model is not equivalent to the original model!"

        # Prepare the model for quantization aware training. This inserts observers in
        # the model that will observe activation tensors during calibration.
    quantized_model = QuantizedAE.QuantizedAE(CAE_encoders.CAE_conv_encoder_arch(16), CAE_decoders.CAE_conv_large_decoder_arch(16))
    
    prepare_dataloader

    # # Use training data for calibration.
    print("Training QAT Model...")
    quantized_model.train()
    train_model(model=quantized_model, train_loader=train_loader, test_loader=test_loader, device=cuda_device, learning_rate=1e-3, num_epochs=10)
    quantized_model.to(cpu_device)

    # Using high-level static quantization wrapper
    # The above steps, including torch.quantization.prepare, calibrate_model, and torch.quantization.convert, are also equivalent to
    # quantized_model = torch.quantization.quantize_qat(model=quantized_model, run_fn=train_model, run_args=[train_loader, test_loader, cuda_device], mapping=None, inplace=False)

    quantized_model = torch.quantization.convert(quantized_model, inplace=True)

    quantized_model.eval()

    # Print quantized model.
    print(quantized_model)

    # Save quantized model.
    save_torchscript_model(model=quantized_model, model_dir=model_dir, model_filename=quantized_model_filename)

    # Load quantized model.
    quantized_jit_model = load_torchscript_model(model_filepath=quantized_model_filepath, device=cpu_device)

    _, int8_eval_accuracy = evaluate_model(model=quantized_jit_model, test_loader=test_loader, device=cpu_device, criterion=None)

    print("INT8 evaluation accuracy: {:.3f}".format(int8_eval_accuracy))

    int8_cpu_inference_latency = measure_inference_latency(model=quantized_model, device=cpu_device, input_size=(1,3,32,32), num_samples=100)
    int8_jit_cpu_inference_latency = measure_inference_latency(model=quantized_jit_model, device=cpu_device, input_size=(1,3,32,32), num_samples=100)
    
    print("FP32 CPU Inference Latency: {:.2f} ms / sample".format(fp32_cpu_inference_latency * 1000))
    print("FP32 CUDA Inference Latency: {:.2f} ms / sample".format(fp32_gpu_inference_latency * 1000))
    print("INT8 CPU Inference Latency: {:.2f} ms / sample".format(int8_cpu_inference_latency * 1000))
    print("INT8 JIT CPU Inference Latency: {:.2f} ms / sample".format(int8_jit_cpu_inference_latency * 1000))

if __name__ == "__main__":

    main()

1


KeyboardInterrupt: 